In [233]:
#import

import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import datetime as dt
from datetime import datetime
from datetime import timedelta
from sklearn.metrics import accuracy_score

In [234]:
# Create pathway to mock data
path = Path("Sample_data.csv")

In [235]:
# load dataframe
crime_df = pd.read_csv(path)
crime_df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,9/5/2015 13:30,043XX S WOOD ST,486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12,61,08B,1165074.0,1875917.0,2015,2/10/2018 15:50,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,9/4/2015 11:30,008XX N CENTRAL AVE,870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29,25,6,1138875.0,1904869.0,2015,2/10/2018 15:50,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,9/1/2018 0:01,082XX S INGLESIDE AVE,810,THEFT,OVER $500,RESIDENCE,False,True,...,8,44,6,NaN,NaN,2018,4/6/2019 16:04,NaN,NaN,NaN
3,10224740,HY411595,9/5/2015 12:45,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35,21,18,1152037.0,1920384.0,2015,2/10/2018 15:50,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,9/5/2015 13:00,0000X N LARAMIE AVE,560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28,25,08A,1141706.0,1900086.0,2015,2/10/2018 15:50,41.881903,-87.755121,"(41.881903443, -87.755121152)"


In [236]:
crime_df = crime_df[crime_df["Year"] >= 2015]

In [237]:
# Taking Count
crime_df.count()

ID                      197
Case Number             197
Date                    197
Block                   197
IUCR                    197
Primary Type            197
Description             197
Location Description    196
Arrest                  197
Domestic                197
Beat                    197
District                197
Ward                    197
Community Area          197
FBI Code                197
X Coordinate            193
Y Coordinate            193
Year                    197
Updated On              197
Latitude                193
Longitude               193
Location                193
dtype: int64

In [238]:
# Drop na values
crime_df = crime_df.dropna()

In [239]:
# Validate drops in na
crime_df.count()

ID                      193
Case Number             193
Date                    193
Block                   193
IUCR                    193
Primary Type            193
Description             193
Location Description    193
Arrest                  193
Domestic                193
Beat                    193
District                193
Ward                    193
Community Area          193
FBI Code                193
X Coordinate            193
Y Coordinate            193
Year                    193
Updated On              193
Latitude                193
Longitude               193
Location                193
dtype: int64

In [240]:
# dropping unnessary columns ( atbitrary placeholders)
cleaned_crime_df = crime_df.drop(columns = ["Case Number", "ID", "IUCR", "Block", "Year", "Updated On", "Description", "FBI Code", "Location"])

In [241]:
#Obtaining the Hour from the time in the Date Column and dropping Date column
cleaned_crime_df["Hour"] = pd.to_datetime(cleaned_crime_df["Date"]).dt.hour
cleaned_crime_df = cleaned_crime_df.drop(columns = ["Date"])

In [242]:
cleaned_crime_df.head()

,Primary Type,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Latitude,Longitude,Hour
0,BATTERY,RESIDENCE,False,True,924,9,12,61,1165074.0,1875917.0,41.815117,-87.670000,13
1,THEFT,CTA BUS,False,False,1511,15,29,25,1138875.0,1904869.0,41.895080,-87.765400,11
3,NARCOTICS,SIDEWALK,True,False,1412,14,35,21,1152037.0,1920384.0,41.937406,-87.716650,12
4,ASSAULT,APARTMENT,False,True,1522,15,28,25,1141706.0,1900086.0,41.881903,-87.755121,13
5,BURGLARY,RESIDENCE,False,False,614,6,21,71,1168430.0,1850165.0,41.744379,-87.658431,10


In [243]:
#Binning the time data
bins = [-1, 3, 7, 11, 15, 19, 24]
cleaned_crime_df['Time_binned'] = pd.cut(cleaned_crime_df['Hour'],
                                         bins=bins, 
                                         labels = [1,2,3,4,5,6])

In [244]:
cleaned_crime_df = cleaned_crime_df.drop(columns = ["Hour"])

In [245]:
cleaned_crime_df.head()

,Primary Type,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Latitude,Longitude,Time_binned
0,BATTERY,RESIDENCE,False,True,924,9,12,61,1165074.0,1875917.0,41.815117,-87.670000,4
1,THEFT,CTA BUS,False,False,1511,15,29,25,1138875.0,1904869.0,41.895080,-87.765400,3
3,NARCOTICS,SIDEWALK,True,False,1412,14,35,21,1152037.0,1920384.0,41.937406,-87.716650,4
4,ASSAULT,APARTMENT,False,True,1522,15,28,25,1141706.0,1900086.0,41.881903,-87.755121,4
5,BURGLARY,RESIDENCE,False,False,614,6,21,71,1168430.0,1850165.0,41.744379,-87.658431,3


In [246]:
#Adding labelencoder for Primaty Type of crime
le = LabelEncoder()
cleaned_crime_df['Primary Type'] = le.fit_transform(cleaned_crime_df['Primary Type'])
cleaned_crime_df['Location Description'] = le.fit_transform(cleaned_crime_df['Location Description'])

In [247]:
cleaned_crime_df.head(10)

,Primary Type,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Latitude,Longitude,Time_binned
0,1,19,False,True,924,9,12,61,1165074.0,1875917.0,41.815117,-87.670000,4
1,14,7,False,False,1511,15,29,25,1138875.0,1904869.0,41.895080,-87.765400,3
3,8,24,True,False,1412,14,35,21,1152037.0,1920384.0,41.937406,-87.716650,4
4,0,1,False,True,1522,15,28,25,1141706.0,1900086.0,41.881903,-87.755121,4
5,2,19,False,False,614,6,21,71,1168430.0,1850165.0,41.744379,-87.658431,3
6,2,21,False,False,1434,14,32,24,1161628.0,1912157.0,41.914636,-87.681631,5
7,14,12,True,False,1034,10,25,31,1159734.0,1889313.0,41.851989,-87.689219,4
8,12,27,False,True,1222,12,27,27,1155536.0,1900515.0,41.882814,-87.704326,3
10,14,18,False,False,833,8,13,65,1150938.0,1857056.0,41.763648,-87.722345,4
11,1,25,False,False,1623,16,45,11,1137969.0,1934340.0,41.975968,-87.768014,3


In [248]:
# Define the features set.
X = cleaned_crime_df.copy()
X = X.drop("Arrest", axis=1)
X.head(10)

,Primary Type,Location Description,Domestic,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Latitude,Longitude,Time_binned
0,1,19,True,924,9,12,61,1165074.0,1875917.0,41.815117,-87.670000,4
1,14,7,False,1511,15,29,25,1138875.0,1904869.0,41.895080,-87.765400,3
3,8,24,False,1412,14,35,21,1152037.0,1920384.0,41.937406,-87.716650,4
4,0,1,True,1522,15,28,25,1141706.0,1900086.0,41.881903,-87.755121,4
5,2,19,False,614,6,21,71,1168430.0,1850165.0,41.744379,-87.658431,3
6,2,21,False,1434,14,32,24,1161628.0,1912157.0,41.914636,-87.681631,5
7,14,12,False,1034,10,25,31,1159734.0,1889313.0,41.851989,-87.689219,4
8,12,27,True,1222,12,27,27,1155536.0,1900515.0,41.882814,-87.704326,3
10,14,18,False,833,8,13,65,1150938.0,1857056.0,41.763648,-87.722345,4
11,1,25,False,1623,16,45,11,1137969.0,1934340.0,41.975968,-87.768014,3


In [249]:
# Define the target set.
y = cleaned_crime_df["Arrest"].ravel()
y[:5]

array([False, False,  True, False, False])

In [250]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.25)

In [251]:
# Creating a StandardScaler
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [252]:
#WARNING: COMPUTATIONALLY HEAVY
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=56, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [253]:
# Making predictions using X_test_scaled
predictions = rf_model.predict(X_test_scaled)
predictions

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
        True, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False,  True, False, False, False,  True, False, False, False,
       False, False,

In [254]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,101,4
Actual 1,33,7
